In [1]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import pickle
import networkx as nx

In [2]:
import obonet
graph = obonet.read_obo('hp.obo')

In [12]:
pheno = pd.read_csv('~/data/pheno.20230209.tsv', sep='\t')

In [13]:
attri = pd.read_csv('metadata_sample_attributes.tsv', sep='\t')

In [112]:
genes = pd.read_csv('Tx.def.tsv', sep = '\t')

In [113]:
individual_level = pd.read_csv('indiv_group.dynamic_deep.tsv', sep = '\t')

In [114]:
individual_level

,#projid,hc_order,original_order,indiv_group
0,50103129,1,287,1
1,20124321,2,131,1
2,21101658,3,217,1
3,65292866,4,354,1
4,20151388,5,137,1
...,...,...,...,...
422,50107583,423,296,13
423,20730959,424,183,13
424,50101785,425,283,13
425,11326252,426,75,15


In [115]:
hpo_df = pd.read_csv('genes_to_phenotype.txt', sep='\t')

In [116]:
ensembl = pd.read_csv('mart_export.txt', sep='\t')
ensembl[ensembl['Gene name'].isin(group_1_genes)]['Gene stable ID'].tolist()

['ENSG00000281919',
 'ENSG00000273936',
 'ENSG00000282379',
 'ENSG00000276836',
 'ENSG00000275867',
 'ENSG00000276559',
 'ENSG00000276907',
 'ENSG00000278458',
 'ENSG00000100346',
 'ENSG00000281626',
 'ENSG00000168546',
 'ENSG00000283589',
 'ENSG00000126790',
 'ENSG00000145536',
 'ENSG00000280962',
 'ENSG00000281244',
 'ENSG00000134779',
 'ENSG00000175985',
 'ENSG00000282854',
 'ENSG00000149654',
 'ENSG00000134013',
 'ENSG00000149050',
 'ENSG00000157542',
 'ENSG00000121691',
 'ENSG00000283985',
 'ENSG00000107614',
 'ENSG00000064687',
 'ENSG00000116017',
 'ENSG00000091656',
 'ENSG00000157927',
 'ENSG00000011132',
 'ENSG00000130702',
 'ENSG00000138061',
 'ENSG00000173638',
 'ENSG00000077327',
 'ENSG00000165973',
 'ENSG00000161082',
 'ENSG00000160193',
 'ENSG00000115163',
 'ENSG00000132541',
 'ENSG00000166012',
 'ENSG00000133740',
 'ENSG00000156928',
 'ENSG00000101844',
 'ENSG00000119547',
 'ENSG00000288490',
 'ENSG00000101638',
 'ENSG00000156469',
 'ENSG00000088325',
 'ENSG00000120992',


In [117]:
hpos = []
gene_ids = []
for i in list(set(individual_level.indiv_group.unique())):
    group_genes = genes[genes['#Tx_idx'] == i]['gene_symbol'].tolist()
    hpo = list(set(hpo_df[hpo_df['gene_symbol'].isin(group_genes)]['hpo_id']))
    gene_id = ensembl[ensembl['Gene name'].isin(group_genes)]['Gene stable ID'].tolist()
    hpos.append(hpo)
    gene_ids.append(gene_id)

In [118]:
df = pd.DataFrame({
    'id': ["Group_1", "Group_2", "Group_3", "Group_4", "Group_5", "Group_6", "Group_7",
          "Group_8", "Group_9", "Group_10", "Group_11", "Group_12", "Group_13", "Group_14", "Group_15"],
    'positive_phenotypes': hpos,
    'all_candidate_genes': gene_ids })

In [119]:
import csv
import json
def row_to_json(row):
    json_dict = {
        'id': row['id'],
        'positive_phenotypes': row['positive_phenotypes'],
        'all_candidate_genes': row['all_candidate_genes']
    }
    return json.dumps(json_dict)

# Write the patient information to a JSONLines file
with open('patients.jsonl', 'w') as f:
    for _, row in df.iterrows():
        f.write(row_to_json(row) + '\n')